# pgvector RAG Implementation Test

**Purpose**: Test pgvector-based RAG before migrating from ChromaDB

**Input Data**: eTMS USER GUIDE DOCUMENT.pdf

**Testing Plan**:
1. Setup PostgreSQL connection with pgvector
2. Extract text from PDF
3. Chunk documents intelligently
4. Generate embeddings using sentence-transformers
5. Store vectors in PostgreSQL
6. Create HNSW index for fast similarity search
7. Test retrieval quality
8. Benchmark performance
9. Compare with ChromaDB approach

**Date**: 2025-11-03

## 1. Install Dependencies

In [1]:
# Install required packages
!pip install psycopg2-binary sentence-transformers pypdf2 numpy pandas tqdm langchain-community

^C


## 2. Setup PostgreSQL Connection

In [18]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import os

# Database connection parameters
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'chatbot_db',  # Update if different
    'user': 'postgres',
    'password': os.getenv('DB_PASSWORD', '123456')  # Use environment variable
}

# Test connection
try:
    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    print("✅ Connected to PostgreSQL")
    
    # Check PostgreSQL version
    with conn.cursor() as cur:
        cur.execute("SELECT version();")
        version = cur.fetchone()[0]
        print(f"PostgreSQL Version: {version[:50]}...")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    raise

✅ Connected to PostgreSQL
PostgreSQL Version: PostgreSQL 18.0 on x86_64-windows, compiled by msv...


## 3. Enable pgvector Extension

In [19]:
# Enable pgvector extension
try:
    with conn.cursor() as cur:
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        print("✅ pgvector extension enabled")
        
        # Verify extension
        cur.execute("SELECT extname, extversion FROM pg_extension WHERE extname = 'vector';")
        result = cur.fetchone()
        if result:
            print(f"pgvector version: {result[1]}")
        else:
            print("⚠️ pgvector not found - may need manual installation")
except Exception as e:
    print(f"❌ pgvector setup failed: {e}")
    print("Install pgvector: https://github.com/pgvector/pgvector#installation")

❌ pgvector setup failed: extension "vector" is not available
HINT:  The extension must first be installed on the system where PostgreSQL is running.

Install pgvector: https://github.com/pgvector/pgvector#installation


## 4. Create Document Table with Vector Column

In [13]:
# Create table for document chunks
# Using 384 dimensions for all-MiniLM-L6-v2 model

CREATE_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS document_chunks (
    chunk_id SERIAL PRIMARY KEY,
    document_name VARCHAR(255) NOT NULL,
    page_number INTEGER,
    chunk_text TEXT NOT NULL,
    chunk_index INTEGER NOT NULL,
    embedding vector(384),  -- 384 dimensions for all-MiniLM-L6-v2
    metadata JSONB,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

try:
    with conn.cursor() as cur:
        # Drop existing table for clean test
        cur.execute("DROP TABLE IF EXISTS document_chunks;")
        print("🗑️ Dropped existing table (clean slate)")
        
        # Create new table
        cur.execute(CREATE_TABLE_SQL)
        print("✅ Created document_chunks table with vector column (384 dimensions)")
except Exception as e:
    print(f"❌ Table creation failed: {e}")
    raise

🗑️ Dropped existing table (clean slate)
❌ Table creation failed: type "vector" does not exist
LINE 8:     embedding vector(384),  -- 384 dimensions for all-MiniLM...
                      ^



UndefinedObject: type "vector" does not exist
LINE 8:     embedding vector(384),  -- 384 dimensions for all-MiniLM...
                      ^


## 5. Extract Text from PDF

In [ ]:
import PyPDF2
from typing import List, Dict

PDF_PATH = "eTMS USER GUIDE DOCUMENT.pdf"

def extract_text_from_pdf(pdf_path: str) -> List[Dict[str, any]]:
    """Extract text from PDF page by page."""
    pages = []
    
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            total_pages = len(pdf_reader.pages)
            
            print(f"📄 Processing {total_pages} pages from {pdf_path}")
            
            for page_num in range(total_pages):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                if text.strip():
                    pages.append({
                        'page_number': page_num + 1,
                        'text': text.strip()
                    })
            
            print(f"✅ Extracted {len(pages)} pages with content")
            return pages
            
    except FileNotFoundError:
        print(f"❌ PDF not found: {pdf_path}")
        raise
    except Exception as e:
        print(f"❌ PDF extraction failed: {e}")
        raise

# Extract text
pages = extract_text_from_pdf(PDF_PATH)

# Display sample
if pages:
    print("\n📄 Sample from first page:")
    print(pages[0]['text'][:300] + "...")

## 6. Chunk Documents & Generate Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

def chunk_text(text: str, chunk_size: int = 500, chunk_overlap: int = 50) -> List[str]:
    """Split text into overlapping chunks."""
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        if chunk.strip():
            chunks.append(chunk.strip())
        start += chunk_size - chunk_overlap
    return chunks

def chunk_pages(pages: List[Dict], chunk_size: int = 500) -> List[Dict]:
    """Chunk all pages with metadata."""
    all_chunks = []
    for page in pages:
        page_chunks = chunk_text(page['text'], chunk_size)
        for idx, chunk in enumerate(page_chunks):
            all_chunks.append({
                'page_number': page['page_number'],
                'chunk_index': idx,
                'text': chunk,
                'metadata': {'page': page['page_number'], 'chunk': idx}
            })
    return all_chunks

# Chunk documents
chunks = chunk_pages(pages)
print(f"✅ Created {len(chunks)} chunks from {len(pages)} pages")

# Load embedding model
print("\n📥 Loading embedding model (all-MiniLM-L6-v2)...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Model loaded")

# Generate embeddings
print(f"\n🔢 Generating embeddings for {len(chunks)} chunks...")
chunk_texts = [chunk['text'] for chunk in chunks]
embeddings = embedding_model.encode(chunk_texts, show_progress_bar=True, convert_to_numpy=True)
print(f"✅ Generated embeddings with shape: {embeddings.shape}")

## 7. Store Vectors in PostgreSQL

In [ ]:
import json
from tqdm import tqdm

DOCUMENT_NAME = "eTMS USER GUIDE DOCUMENT.pdf"

INSERT_SQL = """
INSERT INTO document_chunks (
    document_name, page_number, chunk_text, chunk_index, embedding, metadata
) VALUES (%s, %s, %s, %s, %s, %s)
"""

try:
    with conn.cursor() as cur:
        print(f"💾 Storing {len(chunks)} chunks in database...")
        
        for chunk, embedding in tqdm(zip(chunks, embeddings), total=len(chunks)):
            cur.execute(INSERT_SQL, (
                DOCUMENT_NAME,
                chunk['page_number'],
                chunk['text'],
                chunk['chunk_index'],
                embedding.tolist(),
                json.dumps(chunk['metadata'])
            ))
        
        conn.commit()
        print("✅ All chunks stored successfully")
        
        # Verify count
        cur.execute("SELECT COUNT(*) FROM document_chunks;")
        count = cur.fetchone()[0]
        print(f"📊 Total chunks in database: {count}")
        
except Exception as e:
    conn.rollback()
    print(f"❌ Storage failed: {e}")
    raise

## 8. Create HNSW Index & Test Search

In [ ]:
import time

# Create HNSW index
CREATE_INDEX_SQL = """
CREATE INDEX IF NOT EXISTS document_chunks_embedding_idx
ON document_chunks
USING hnsw (embedding vector_cosine_ops)
WITH (m = 16, ef_construction = 64);
"""

print("🔨 Creating HNSW index...")
start_time = time.time()
with conn.cursor() as cur:
    cur.execute(CREATE_INDEX_SQL)
elapsed = time.time() - start_time
print(f"✅ HNSW index created in {elapsed:.2f} seconds")

# Test search function
def search_similar_chunks(conn, query: str, embedding_model, top_k: int = 5) -> List[Dict]:
    """Search for similar document chunks."""
    query_embedding = embedding_model.encode([query])[0].tolist()
    
    SEARCH_SQL = """
    SELECT chunk_id, document_name, page_number, chunk_text, chunk_index, metadata,
           1 - (embedding <=> %s::vector) AS similarity_score
    FROM document_chunks
    ORDER BY embedding <=> %s::vector
    LIMIT %s;
    """
    
    with conn.cursor() as cur:
        cur.execute(SEARCH_SQL, (query_embedding, query_embedding, top_k))
        results = cur.fetchall()
        
        return [{
            'chunk_id': row[0],
            'document_name': row[1],
            'page_number': row[2],
            'text': row[3],
            'chunk_index': row[4],
            'metadata': row[5],
            'similarity_score': float(row[6])
        } for row in results]

# Test queries
test_queries = [
    "How do I track shipments?",
    "What is the login process?",
    "How to generate reports?"
]

print("\n🔍 Testing similarity search...\n")
for query in test_queries:
    print(f"Query: '{query}'")
    print("-" * 80)
    
    start_time = time.time()
    results = search_similar_chunks(conn, query, embedding_model, top_k=3)
    search_time = time.time() - start_time
    
    print(f"⏱️ Search time: {search_time*1000:.2f}ms\n")
    
    for i, result in enumerate(results, 1):
        print(f"Result {i} (Score: {result['similarity_score']:.4f})")
        print(f"Page {result['page_number']}, Chunk {result['chunk_index']}")
        print(f"Text: {result['text'][:150]}...")
        print()
    
    print("=" * 80 + "\n")

## 9. Summary & Next Steps

In [ ]:
print("="*80)
print("📊 pgvector RAG Test Summary")
print("="*80)

with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM document_chunks;")
    total_chunks = cur.fetchone()[0]
    
    cur.execute("SELECT COUNT(DISTINCT page_number) FROM document_chunks;")
    total_pages = cur.fetchone()[0]
    
    print(f"\n📄 Document Statistics:")
    print(f"  - Total Pages: {total_pages}")
    print(f"  - Total Chunks: {total_chunks}")
    print(f"  - Avg Chunks/Page: {total_chunks/total_pages:.1f}")
    
    print(f"\n✅ Status: pgvector RAG test successful!")
    print(f"\n📋 Next Steps:")
    print(f"  1. Review test results and performance")
    print(f"  2. Follow full migration plan in PGVECTOR_MIGRATION_PLAN.md")
    print(f"  3. Migrate existing ChromaDB data to pgvector")
    print(f"  4. Update chatbot agents to use pgvector")
    print(f"  5. Test in production environment")

# Close connection
conn.close()
print("\n✅ Database connection closed")